In [1]:
%cd ..

/Users/treycole/Codes/WanPy


In [2]:
%load_ext line_profiler
%timeit

In [3]:
from WanPy.WanPy import *
import WanPy.models as models
import WanPy.plotting as plot

from pythtb import *
import numba
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sympy as sp 
import scipy

In [4]:
delta = 1
t = -1
t2 = 0.2

model = models.Haldane(delta, t, t2).make_supercell([[2,0], [0,2]])

orbs = model.get_orb()
n_orb = model.get_num_orbitals()
n_occ = int(n_orb/2)
lat_vecs = model.get_lat() # lattice vectors

low_E_sites = np.arange(0, model.get_num_orbitals(), 2)
high_E_sites = np.arange(1, model.get_num_orbitals(), 2)

In [5]:
nkx = 10
nky = 10
Nk = nkx*nky

k_mesh = gen_k_mesh(nkx, nky, flat=False, endpoint=False)
k_mesh_flat = gen_k_mesh(nkx, nky, flat=True, endpoint=False)

u_wfs_Wan = wf_array(model, [nkx, nky])
for i in range(k_mesh.shape[0]):
    for j in range(k_mesh.shape[1]):
        u_wfs_Wan.solve_on_one_point(k_mesh[i,j], [i,j])
psi_wfs_Wan = get_bloch_wfs(orbs, u_wfs_Wan, k_mesh)

In [6]:
omit_sites = 4
tf_list = list(np.setdiff1d(low_E_sites, [omit_sites])) # delta on lower energy sites omitting the last site

W0, psi_til_wan = Wannierize(orbs, u_wfs_Wan, tf_list, ret_psi_til=True)
u_tilde_wan = get_bloch_wfs(orbs, psi_til_wan, k_mesh, inverse=True)
M = k_overlap_mat(lat_vecs, orbs, u_tilde_wan)  # [kx, ky, b, m, n]

In [7]:
spread, expc_rsq, expc_r_sq = spread_recip(lat_vecs, M, decomp=True)

print("After first projection")
print(rf"Spread = {spread[0]}")
print(rf"Omega_I = {spread[1]}")
print(rf"Omega_til = {spread[2]}")

After first projection
Spread = [0.22740033 0.22740033 0.22740033]
Omega_I = 0.527173883537877
Omega_til = 0.1550271029866458


In [8]:
# outer window of entangled bands is full occupied manifold
outer_states = u_wfs_Wan._wfs[..., :n_occ, :]

In [9]:
W0_max_loc, Wf_cntrs = max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, 
        iter_num_omega_i=1000, iter_num_omega_til=100,
        Wan_idxs=None, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3
        )

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

In [20]:
%lprun -f max_loc_Wan max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=100, iter_num_omega_til=100, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.13812 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: max_loc_Wan at line 874

Line #      Hits         Time  Per Hit   % Time  Line Contents
   874                                           def max_loc_Wan(
   875                                               lat_vecs,
   876                                               orbs,
   877                                               u_wfs,
   878                                               tf_list,
   879                                               outer_states,
   880                                               iter_num_omega_i=1000,
   881                                               iter_num_omega_til=1000,
   882                                               eps=1e-3,
   883                                               tol=1e-10,
   884                                               alpha=1,
   885                                               Wan_idxs=None,
   886                

In [11]:
%lprun -f max_loc_Wan max_loc_Wan(lat_vecs, orbs, u_wfs_Wan, tf_list, outer_states, iter_num_omega_i=100, iter_num_omega_til=100, verbose=True, return_uwfs=False, return_wf_centers=True, eps=1e-3)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.42204 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: max_loc_Wan at line 904

Line #      Hits         Time  Per Hit   % Time  Line Contents
   904                                           def max_loc_Wan(
   905                                               lat_vecs,
   906                                               orbs,
   907                                               u_wfs,
   908                                               tf_list,
   909                                               outer_states,
   910                                               iter_num_omega_i=1000,
   911                                               iter_num_omega_til=1000,
   912                                               eps=1e-3,
   913                                               tol=1e-17,
   914                                               alpha=1,
   915                                               Wan_idxs=None,
   916                

In [18]:
%lprun -f find_min_unitary find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=1000, verbose=True, tol=1e-17)

0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872957294, Grad mag: 4.700296710623303
2 Omega_til = 0.15471152306816785, Grad mag: 4.674373024957893
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703756
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455415
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243344872213
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646296
7 Omega_til = 0.15431133725558602, Grad mag: 4.546372986410781
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698652
9 Omega_til = 0.15418296687107264, Grad mag: 4.495953588049208
10 Omega_til = 0.15412405839789625, Grad mag: 4.470915467347875
11 Omega_til = 0.15406827955937186, Grad mag: 4.4459928434749125
12 Omega_til = 0.15401537679377503, Grad mag: 4.42118643433021
13 Omega_til = 0.15396512238656693, Grad mag: 4.396496873293546
14 Omega_til = 0.15391731147639431, Grad mag: 4.371924715173343
15 Omega_til = 0.15387175943563747, Grad mag: 4.347470

Timer unit: 1e-09 s

Total time: 0.824593 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_min_unitary at line 784

Line #      Hits         Time  Per Hit   % Time  Line Contents
   784                                           def find_min_unitary(u_wfs, lat_vecs, orbs, eps=1 / 160, iter_num=10, verbose=False, tol=1e-12):
   785                                               """
   786                                               Finds the unitary that minimizing the gauge dependent part of the spread. 
   787                                           
   788                                               Args:
   789                                                   lat_vecs: Lattice vectors
   790                                                   M: Overlap matrix
   791                                                   eps: Step size for gradient descent
   792                                                   iter_num: Number of iterations
   793                   

In [13]:
%lprun -f find_optimal_subspace find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=100, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
10 Omega_I: 0.48867378673602585
11 Omega_I: 0.48578198905465403
12 Omega_I: 0.48301327418404705
13 Omega_I: 0.4803737222134296
14 Omega_I: 0.47786681964443983
15 Omega_I: 0.475493746569461
16 Omega_I: 0.4732537429670067
17 Omega_I: 0.4711444912243334
18 Omega_I: 0.46916247631807456
19 Omega_I: 0.46730330365649536
20 Omega_I: 0.4655619672654045
21 Omega_I: 0.46393306868567136
22 Omega_I: 0.4624109910153313
23 Omega_I: 0.46099003422475665
24 Omega_I: 0.45966451817546417
25 Omega_I: 0.45842885934531563
26 Omega_I: 0.45727762651902565
27 Omega_I: 0.45620557987733384
28 Omega_I: 0.4552076971311633
29 Omega_I: 0.4542791896506486
30 Omega_I: 0.45341551094840576
31 Omega_I: 0.45261235938839023
32 Om

Timer unit: 1e-09 s

Total time: 1.05762 s
File: /Users/treycole/Codes/WanPy/WanPy/WanPy.py
Function: find_optimal_subspace at line 675

Line #      Hits         Time  Per Hit   % Time  Line Contents
   675                                           def find_optimal_subspace(
   676                                               lat_vecs, orbs, outer_states, inner_states, iter_num=100, verbose=False, tol=1e-17, alpha=1
   677                                           ):
   678         1      10000.0  10000.0      0.0      if isinstance(inner_states, wf_array):
   679                                                   shape = inner_states._wfs.shape  # [*nks, idx, orb]
   680                                                   inner_states = np.array(inner_states._wfs)
   681                                               else:
   682         1      25000.0  25000.0      0.0          shape = inner_states.shape  # [*nks, idx, orb]
   683                                           
   684       

In [13]:
%timeit find_min_unitary(u_tilde_wan, lat_vecs, orbs, eps=1e-3, iter_num=10, verbose=True, tol=1e-17)

0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872957294, Grad mag: 4.700296710623303
2 Omega_til = 0.15471152306816785, Grad mag: 4.674373024957893
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703756
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455415
5 Omega_til = 0.1544560793920921, Grad mag: 4.597243344872213
6 Omega_til = 0.1543814689220185, Grad mag: 4.571752344646296
7 Omega_til = 0.15431133725558602, Grad mag: 4.546372986410781
8 Omega_til = 0.1542452878134296, Grad mag: 4.521106396698652
9 Omega_til = 0.15418296687107264, Grad mag: 4.495953588049208
0 Omega_til = 0.15491402050243544, Grad mag: 4.726324345017292
1 Omega_til = 0.15480909872957294, Grad mag: 4.700296710623303
2 Omega_til = 0.15471152306816785, Grad mag: 4.674373024957893
3 Omega_til = 0.15462057378786062, Grad mag: 4.648555139703756
4 Omega_til = 0.1545356140277706, Grad mag: 4.622844737455415
5 Omega_til = 0.1544560793920921, Grad mag: 4.59724334487221

In [19]:
%timeit find_optimal_subspace(lat_vecs, orbs, outer_states, u_tilde_wan, iter_num=10, verbose=True)

0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: 0.5114317051940603
4 Omega_I: 0.5079889673197662
5 Omega_I: 0.504598731828422
6 Omega_I: 0.5012619108799339
7 Omega_I: 0.49798763126911627
8 Omega_I: 0.4947889209823177
9 Omega_I: 0.4916798757270802
0 Omega_I: 0.5224352455932965
1 Omega_I: 0.5185655608322894
2 Omega_I: 0.5149416822333317
3 Omega_I: